# Ejemplo Hola Mundo GPU

El objetivo del ejemplo es explicar como debaguiar por código la ejecución del kernel que ejecuta en el co-procesador gráfico.

# Listar el tipo de GPU que esta activa en el cuaderno:

In [111]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-c73adb37-61c3-0a97-6d6c-e1098bcdfcb2)


# Listar máxima cantidad de Grillas /Bloques soportados por el GPU:

In [112]:
!cd  /usr/local/cuda/samples/1_Utilities/deviceQuery/; make >/dev/null
!/usr/local/cuda/samples/1_Utilities/deviceQuery/deviceQuery | grep "Max dimension"

  Max dimension size of a thread block (x,y,z): (1024, 1024, 64)
  Max dimension size of a grid size    (x,y,z): (2147483647, 65535, 65535)


# Ejemplo Hola Mundo con GPU.

Reseteo Buffer de plataforma colab de la saludar del GPU.

In [105]:
!>/var/colab/app.log

Ejecuto el ejemplo, donde se ve el kernel suporta la funcion printf().

In [107]:
#!/usr/bin/env python
# --------------------------------------------
#@title Parámetros de ejecución { vertical-output: true }

cantidad_N =   50#@param {type: "number"}
# --------------------------------------------
import numpy
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

pycuda.tools.make_default_context()

# CPU - Defino la función kernel que ejecutará en GPU.
module = SourceModule("""
#include <stdio.h>
__global__ void kernel_HolaMundo( int N )
{
  int idx = threadIdx.x + blockIdx.x*blockDim.x;

  if( idx < N )
  {
    printf( "Hola Mundo desde el GPU - idx %d, Bloque id %d, Thread id %d \\n ", idx, blockIdx.x, threadIdx.x );
  }
  else
  {
    printf( "No saludo, porque soy un hilo planificado de mas - idx %d \\n ", idx );
  }

}
""")

# CPU - Genero la función kernel.
kernel = module.get_function("kernel_HolaMundo")

dim_hilo = 32
dim_bloque = numpy.int( (cantidad_N+dim_hilo-1) / dim_hilo )

#TODO: Ojo, con los tipos de las variables en el kernel.
kernel( numpy.int32(cantidad_N),  block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )

cuda.Context.synchronize()


print( "Hola Mundo desde el CPU => Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )



Hola Mundo desde el CPU => Thread x:  32 , Bloque x: 2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: The CUDA compiler succeeded, but said the following:
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).



Muestro el buffer de COLAB: 

In [108]:
cat /var/colab/app.log

{"pid":1,"type":"jupyter","level":40,"msg":"Hola Mundo desde el GPU - idx 32, Bloque id 1, Thread id 0 ","time":"2021-06-23T14:16:17.842Z","v":0}
{"pid":1,"type":"jupyter","level":40,"msg":" Hola Mundo desde el GPU - idx 33, Bloque id 1, Thread id 1 ","time":"2021-06-23T14:16:17.842Z","v":0}
{"pid":1,"type":"jupyter","level":40,"msg":" Hola Mundo desde el GPU - idx 34, Bloque id 1, Thread id 2 ","time":"2021-06-23T14:16:17.842Z","v":0}
{"pid":1,"type":"jupyter","level":40,"msg":" Hola Mundo desde el GPU - idx 35, Bloque id 1, Thread id 3 ","time":"2021-06-23T14:16:17.842Z","v":0}
{"pid":1,"type":"jupyter","level":40,"msg":" Hola Mundo desde el GPU - idx 36, Bloque id 1, Thread id 4 ","time":"2021-06-23T14:16:17.842Z","v":0}
{"pid":1,"type":"jupyter","level":40,"msg":" Hola Mundo desde el GPU - idx 37, Bloque id 1, Thread id 5 ","time":"2021-06-23T14:16:17.843Z","v":0}
{"pid":1,"type":"jupyter","level":40,"msg":" Hola Mundo desde el GPU - idx 38, Bloque id 1, Thread id 6 ","time":"2021-

# Debug con  CUDA.

Ver el ejemplo desde: 
https://wiki.tiker.net/PyCuda/FrequentlyAskedQuestions/#system-specific-questions


In [110]:
!cuda-gdb 

NVIDIA (R) CUDA Debugger
11.0 release
Portions Copyright (C) 2007-2020 NVIDIA Corporation
GNU gdb (GDB) 8.2
Copyright (C) 2018 Free Software Foundation, Inc.
License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.
Type "show copying" and "show warranty" for details.
This GDB was configured as "x86_64-pc-linux-gnu".
Type "show configuration" for configuration details.
For bug reporting instructions, please see:
<http://www.gnu.org/software/gdb/bugs/>.
Find the GDB manual and other documentation resources online at:
    <http://www.gnu.org/software/gdb/documentation/>.

For help, type "help".
Type "apropos word" to search for commands related to "word".
(cuda-gdb) help
List of classes of commands:

aliases -- Aliases of other commands
breakpoints -- Making program stop at certain points
cuda  -- CUDA commands
data -- Examining data
files -- Specif